# Capstone Project - The Battle of the Neighborhoods (Week 4 & 5)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Similarity of Neighborhoods](#introduction)
* [Data](#data)

## Introduction: Similarity of Neighborhoods <a name="introduction"></a>

In this project, we will try to make comparison on neighborhoods of several major financial capitals. Inspired by the question asked in the Week 4 description, namely the similarity or dissimilarity of cities, we will try to group the neighborhoods and boroughs over different cities, as a preliminary attempt to identify the functionality and types of them. My hope it that such categorization will shed light on the design of cities of similar type in the future and on relevant studies of developments and design of cities. 

## Data <a name="data"></a>

Aside from a couple of tables, which we will obtain by scraping some webpages, we will also make use of **geopy** for the geospatial location of neighboorhoods. We will also access information of venues using **Foursquare API**.

First, let's load up the libraries. lxml is used for parsing the html content involved in table wrangling. geopy is used to obtain geospatial location from address. Also, we will use K Means clustering.

In [1]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


alta

In the wikipedia page below, I found the table for the 20 districts of Paris. Let's wrangle it. (Note: 20 is a somewhat smaller number of districts as compared to New York data, although comparable to Toronto. However, with the attempt I made, I could not find any more detailed infos on the smaller administritive zones. Please let me know in case you know where to get it. As a preliminary attempt, we are good with 20 districts.)

In [68]:
url = 'https://en.wikipedia.org/wiki/Arrondissements_of_Paris'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Through trial and error, we get to know that the 15-34 entries are the "Arrondissements"(French word for county or district) we need. And we need only the names for future location checking. We will make a list of these names.

In [3]:
paris_dist = []
for x in range(15,35):
    paris_dist.append(tr_elements[x].text_content().split('\n')[2])
print(paris_dist)

['Louvre', 'Bourse', 'Temple', 'Hôtel-de-Ville', 'Panthéon', 'Luxembourg', 'Palais-Bourbon', 'Élysée', 'Opéra', 'Entrepôt', 'Popincourt', 'Reuilly', 'Gobelins', 'Observatoire', 'Vaugirard', 'Passy', 'Batignolles-Monceau', 'Butte-Montmartre', 'Buttes-Chaumont', 'Ménilmontant']


Now we will obtain the geospatial location from the district names using geopy. 

In [4]:
suffix = ', Paris, France'
lats = []
longs = []

geolocator = Nominatim(user_agent="ny_explorer")
for i in range(len(paris_dist)):
    location = geolocator.geocode(paris_dist[i] + suffix)
    latitude = location.latitude
    longitude = location.longitude
    lats.append(latitude)
    longs.append(longitude)
print(lats,longs)

[48.8611473, 48.8686296, 48.8665004, 48.856426299999995, 48.84619085, 48.8504333, 48.86159615, 48.8466437, 48.8706446, 48.876106, 48.858416, 48.8396154, 48.8323973, 48.8295667, 48.8413705, 48.8575047, 48.881452, 48.8900117, 48.8783961, 48.8667079] [2.33802768704666, 2.3414739, 2.360708, 2.3525275780116073, 2.346078521905153, 2.3329507, 2.3179092733655935, 2.3698297, 2.33233, 2.35991, 2.379703, 2.3957517, 2.3555829, 2.3239624642685364, 2.3003827, 2.2809828, 2.3166666, 2.3464668, 2.3812008, 2.3833739]


Make a dataframe. **(Note: 'Neighborhood' column will be named 'Hood' in all three dataframe (Paris, Toronto, and NY) involved in this project. Because I found that there are venues in Toronto and New York categorized as 'Neighborhood' (returnd by Foursquare queries), making it a problem joining the dataframes correctly after one-hot encoding.)**

In [5]:
paris_dict = {'Hood': paris_dist, 'Latitude': lats, 'Longitude':longs}
df_paris = pd.DataFrame(paris_dict)
df_paris

,Hood,Latitude,Longitude
0,Louvre,48.861147,2.338028
1,Bourse,48.868630,2.341474
2,Temple,48.866500,2.360708
3,Hôtel-de-Ville,48.856426,2.352528
4,Panthéon,48.846191,2.346079
5,Luxembourg,48.850433,2.332951
6,Palais-Bourbon,48.861596,2.317909
7,Élysée,48.846644,2.369830
8,Opéra,48.870645,2.332330
9,Entrepôt,48.876106,2.359910


Let us also test the folium map by visualizing Paris and its arrondissements.

In [6]:
address = 'Paris, France'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [7]:
# create map of New York using latitude and longitude values
map_paris = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_paris['Latitude'], df_paris['Longitude'], df_paris['Hood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

Now we will tackle the Toronto table. The process is similar to Week 3's assignment. 

In [8]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [9]:
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().replace('\n','')
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code"
2:"Borough"
3:"Neighborhood"


In [10]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().replace('\n','')
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

What the last part does, is that it builds up a list of 3 tuples, each of which contains the name of a column, and a list that holds the content of the column in postal code table.

In [11]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
# Somehow the last row of [[],['Canadian Postal Code'], []] is always included. 
# Couldn't figure out the cause, just deleted it directly.
df = df[:-1]
msk = (df.Borough == 'Not assigned')
df = df[~msk]
df.reset_index(drop=True)
df.rename(columns={'Neighborhood':'Hood'},inplace=True)

Again, we take only the assigned rows as we are only interested in the boroughs. Again, 'Neighborhood' is renamed as 'Hood', due to the potential issue that we already mentioned. 

In [12]:
ll = pd.read_csv("https://cocl.us/Geospatial_data")
df = df.merge(ll, on='Postal Code')
msk = []
for x in df['Borough']:
    msk.append('Toronto' in x)
msk
df_toronto = df[msk]
df_toronto.reset_index(drop=True)

,Postal Code,Borough,Hood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


Similar things will be done to get New York data. We will fetch the data for the whole NYC instead of just Manhattan.

In [13]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [14]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Hood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Hood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Again, 'Neighborhood' is named 'Hood'.

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
# Change the name
df_newyork = neighborhoods

The dataframe has 5 boroughs and 306 neighborhoods.


In [16]:
df_newyork.head()

,Borough,Hood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Now, let's set up Foursquare credentials. 

In [17]:
CLIENT_ID = 'VL3S4DCT1KRZF3FTAC2JZLNZYQLOHPQ4HMEEITVKAMZRMMOZ'
CLIENT_SECRET = 'QLJMPTMTLO5FUEYY4CYS22BEGVHUMWNFANPKCGRQYE4YXYQB'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VL3S4DCT1KRZF3FTAC2JZLNZYQLOHPQ4HMEEITVKAMZRMMOZ
CLIENT_SECRET:QLJMPTMTLO5FUEYY4CYS22BEGVHUMWNFANPKCGRQYE4YXYQB


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We will get venue informations by querying Foursquare database. **(Note that this part takes time. And due to the limitation on the account, it'd be nice to debug with only Toronto data or Paris data before we go all in and load the  New York data. New York data requires a lot more quieries. With a free account we can do the following for only once.)** 

In [19]:
toronto_venues = getNearbyVenues(names=df_toronto['Hood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
paris_venues = getNearbyVenues(names=df_paris['Hood'],
                                   latitudes=df_paris['Latitude'],
                                   longitudes=df_paris['Longitude']
                                  )
newyork_venues = getNearbyVenues(names=df_newyork['Hood'],
                                   latitudes=df_newyork['Latitude'],
                                   longitudes=df_newyork['Longitude']
                                  )
print('Done loading venues.')

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [20]:
toronto_venues.to_csv(r'toronto_venues.csv', index = False)
paris_venues.to_csv(r'paris_venues.csv', index = False)
newyork_venues.to_csv(r'newyork_venues.csv', index = False)
print('Venue files saved.')

Venue files saved.


In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1605, 7)


,Hood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [22]:
print(paris_venues.shape)
paris_venues.head()

(1308, 7)


,Hood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Louvre,48.861147,2.338028,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
1,Louvre,48.861147,2.338028,Musée du Louvre,48.860847,2.336440,Art Museum
2,Louvre,48.861147,2.338028,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
3,Louvre,48.861147,2.338028,Place du Palais Royal,48.862523,2.336688,Plaza
4,Louvre,48.861147,2.338028,Palais Royal,48.863236,2.337127,Historic Site


In [23]:
print(newyork_venues.shape)
newyork_venues.head()

(9866, 7)


,Hood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's also make sure that 'Neighborhood' is not in the column names any more. 

In [24]:
print('Neighborhood' in toronto_venues.columns.to_list())
print('Neighborhood' in paris_venues.columns.to_list())
print('Neighborhood' in newyork_venues.columns.to_list())

False
False
False


In [25]:
print('There are {} uniques categories in Toronto venues.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques categories in Paris venues.'.format(len(paris_venues['Venue Category'].unique())))
print('There are {} uniques categories in New York venues.'.format(len(newyork_venues['Venue Category'].unique())))

There are 235 uniques categories in Toronto venues.
There are 210 uniques categories in Paris venues.
There are 430 uniques categories in New York venues.
